In [1]:
# 🔹 Import required libraries
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages   # reducer for messages
from langgraph.checkpoint.memory import MemorySaver   # for persistence (RAM-based)
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_groq import ChatGroq   #  using Groq 

In [2]:
# -------------------------------
# Step 1: Define State
# -------------------------------
class ChatState(TypedDict):
    # State will hold conversation history
    messages: Annotated[list[BaseMessage], add_messages]

In [10]:
# Create LLM (Groq)
llm = ChatGroq(model="deepseek-r1-distill-llama-70b", temperature=0.7)

In [11]:
def chat_node(state: ChatState):
    # Take existing conversation history (messages)
    messages = state["messages"]
    
    # Pass them to Groq LLM
    response = llm.invoke(messages)
    
    # Return AI's reply inside list (so it merges into state)
    return {"messages": [response]}

In [12]:
graph = StateGraph(ChatState)
# Add single node "chat"
graph.add_node("chat", chat_node)

In [13]:
# Define workflow edges
graph.add_edge(START, "chat")
graph.add_edge("chat", END)

In [14]:
# -------------------------------
# Step 4: Enable Persistence
# -------------------------------
checkpointer = MemorySaver()   # stores history in RAM
chatbot = graph.compile(checkpointer=checkpointer)

In [15]:
# -------------------------------
# Step 5: Run Chatbot in Loop
# -------------------------------
config = {"configurable": {"thread_id": "rahul_chat"}}  # unique per user

In [16]:
while True:
    user_input = input("You: ")
    
    # Exit condition
    if user_input.strip().lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye! 👋")
        break
    
    # Invoke chatbot
    result = chatbot.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config
    )
    
    # Extract last AI message
    ai_reply = result["messages"][-1].content
    print("Chatbot:", ai_reply)

Chatbot: <think>

</think>

Hello! How can I assist you today? 😊
Chatbot: <think>
Okay, the user just said, "my name is Rahul." I should respond in a friendly and welcoming manner. I want to make sure they feel comfortable and know I'm here to help. Maybe I can greet them by their name and offer assistance. I should keep it simple and open-ended so they feel encouraged to ask for what they need. Let's see, something like, "Hello Rahul! How can I assist you today?" That sounds good. It's polite and opens the door for them to continue the conversation.
</think>

Hello Rahul! How can I assist you today?
Chatbot: Goodbye! 👋
